This notebook produces the final dataset for predicting February results. The results will be for the actual DMC submission.

In [2]:
import pandas as pd
import os
import numpy as np
import pickle

from measures import wrangling

# Import dataset and flatten for neural network input

In [3]:
dataset = pd.read_csv('./data/clean/data_v0.1.csv', header=0)

In [4]:
cols = pd.DataFrame([dataset.columns.tolist()])
cols.to_csv('./data/clean/data_v0.1_columns.csv')

February price columns no longer removed; we need them to make predictions

In [9]:
# Drop February price-columns
# dataset = wrangling.remove_price_cols(dataset, '2018-02-01', '2018-02-28')
print(dataset.columns)
print(dataset.columns[134:134+28]) # February price columns

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'stock',
       ...
       '2018-01-22_sales', '2018-01-23_sales', '2018-01-24_sales',
       '2018-01-25_sales', '2018-01-26_sales', '2018-01-27_sales',
       '2018-01-28_sales', '2018-01-29_sales', '2018-01-30_sales',
       '2018-01-31_sales'],
      dtype='object', length=285)
Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'stock', 'releaseDate', '2017-10-01_price',
       '2017-10-02_price', '2017-10-03_price'],
      dtype='object')
Index(['2018-02-01_price', '2018-02-02_price', '2018-02-03_price',
       '2018-02-04_price', '2018-02-05_price', '2018-02-06_price',
       '2018-02-07_price', '2018-02-08_price', '2018-02-09_price',
       '2018-02-10_price', '2018-02-11_price', '2018-02-12_price',
       '2018-02-13_price', '2018-02-14_price', '2018-02-15_price',
       '2018-02-16_price', '2018-02-17_price', '2018

In [26]:
# Separate Y labels from X variables
Y_dataset = wrangling.sales_cols(dataset, '2017-10-01', '2018-01-31') 
X_dataset = wrangling.remove_sales_cols(dataset, '2017-10-01', '2018-01-31')

Dataset no longer split to big/small clusters

In [23]:
# Y_dataset['total_sales'] = Y_dataset.iloc[:,1:93].sum(axis=1) # [92] = 2017-12-31
# Y_dataset['cluster'] = np.where(Y_dataset['total_sales'] >= 13, 'big', 'small')
# len(Y_dataset[Y_dataset['cluster'] == 'big']) # Should be 2,907 rows
# Y_dataset.to_csv('./data/clean/data_v0.2_sales.csv', index=False)

In [27]:
X_dataset = X_dataset.drop('stock', axis=1)
print(X_dataset.columns)

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'releaseDate',
       ...
       '2018-02-19_price', '2018-02-20_price', '2018-02-21_price',
       '2018-02-22_price', '2018-02-23_price', '2018-02-24_price',
       '2018-02-25_price', '2018-02-26_price', '2018-02-27_price',
       '2018-02-28_price'],
      dtype='object', length=161)


Without February data, we have 12,824x123 = 1,577,352 rows when dataset is flattened. 
With February data, we expect 12,824x(123+28) = 1,936,424 rows when dataset is flattened

In [29]:
# Flatten X so that each row is 1 day
cols = ['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory', 'category', 'subCategory', 'releaseDate']
X_flat = pd.melt(X_dataset, id_vars=cols, var_name='date', value_name='price')
X_flat = X_flat.sort_values(['key', 'date']).reset_index(drop=True)
print(X_flat.shape) # Should be 1,936,424 rows
print(X_flat.tail()) # Quick check of the result

(1936424, 12)
            key  pid_x size_x color   brand     rrp  mainCategory  category  \
1936419  22881S  22881      S  grau  adidas  139.63             1         7   
1936420  22881S  22881      S  grau  adidas  139.63             1         7   
1936421  22881S  22881      S  grau  adidas  139.63             1         7   
1936422  22881S  22881      S  grau  adidas  139.63             1         7   
1936423  22881S  22881      S  grau  adidas  139.63             1         7   

         subCategory releaseDate              date   price  
1936419         31.0  2017-10-01  2018-02-24_price  104.72  
1936420         31.0  2017-10-01  2018-02-25_price  104.72  
1936421         31.0  2017-10-01  2018-02-26_price  104.72  
1936422         31.0  2017-10-01  2018-02-27_price  104.72  
1936423         31.0  2017-10-01  2018-02-28_price  104.72  


In [30]:
# Create February columns for Y containing NA values
for day in range(1, 29):
    if day < 10:
        col_name = '2018-02-0{}_sales'.format(str(day))
    else:
        col_name = '2018-02-{}_sales'.format(str(day))
    Y_dataset[col_name] = np.nan # Because we don't have true Y (sales) values for February
print(Y_dataset.columns[-30:]) # Quick check
print(Y_dataset.tail())

Index(['2018-01-30_sales', '2018-01-31_sales', '2018-02-01_sales',
       '2018-02-02_sales', '2018-02-03_sales', '2018-02-04_sales',
       '2018-02-05_sales', '2018-02-06_sales', '2018-02-07_sales',
       '2018-02-08_sales', '2018-02-09_sales', '2018-02-10_sales',
       '2018-02-11_sales', '2018-02-12_sales', '2018-02-13_sales',
       '2018-02-14_sales', '2018-02-15_sales', '2018-02-16_sales',
       '2018-02-17_sales', '2018-02-18_sales', '2018-02-19_sales',
       '2018-02-20_sales', '2018-02-21_sales', '2018-02-22_sales',
       '2018-02-23_sales', '2018-02-24_sales', '2018-02-25_sales',
       '2018-02-26_sales', '2018-02-27_sales', '2018-02-28_sales'],
      dtype='object')
            key  2017-10-01_sales  2017-10-02_sales  2017-10-03_sales  \
12819  228782XL               0.0               0.0               0.0   
12820    22878L               0.0               0.0               0.0   
12821    22878M               0.0               0.0               0.0   
12822   22878XL

In [31]:
# Flatten Y similarly, so that the rows of Y correspond to that of X
Y_flat = pd.melt(Y_dataset, id_vars='key', var_name='date', value_name='sales')
Y_flat = Y_flat.sort_values(['key', 'date']).reset_index(drop=True)
print(Y_flat.shape) # Should have also 1,936,424 rows
print(Y_flat.tail()) 

(1936424, 3)
            key              date  sales
1936419  22881S  2018-02-24_sales    NaN
1936420  22881S  2018-02-25_sales    NaN
1936421  22881S  2018-02-26_sales    NaN
1936422  22881S  2018-02-27_sales    NaN
1936423  22881S  2018-02-28_sales    NaN


# Cleaning data

In [33]:
# Clean 'date' columns to keep only YYYY-MM-DD part
X_flat['date'] = X_flat['date'].str[0:10]
Y_flat['date'] = Y_flat['date'].str[0:10]

In [34]:
X_flat['subCategory'] = X_flat['subCategory'].fillna(0) # Fill blank sub-category with 0
X_flat['size_x'] = X_flat['size_x'].fillna('NA') # Fill blank sizes with 'NA' string
X_flat['price'] = X_flat['price'].fillna(method='bfill') # Fill blank prices with earliest given price

# Feature engineering

In [35]:
print(X_flat.head())

                   key  pid_x          size_x  color brand    rrp  \
0  10000XL ( 158-170 )  10000  XL ( 158-170 )  gruen  Nike  25.33   
1  10000XL ( 158-170 )  10000  XL ( 158-170 )  gruen  Nike  25.33   
2  10000XL ( 158-170 )  10000  XL ( 158-170 )  gruen  Nike  25.33   
3  10000XL ( 158-170 )  10000  XL ( 158-170 )  gruen  Nike  25.33   
4  10000XL ( 158-170 )  10000  XL ( 158-170 )  gruen  Nike  25.33   

   mainCategory  category  subCategory releaseDate        date  price  
0             1         7         25.0  2017-10-01  2017-10-01  18.99  
1             1         7         25.0  2017-10-01  2017-10-02  18.99  
2             1         7         25.0  2017-10-01  2017-10-03  18.99  
3             1         7         25.0  2017-10-01  2017-10-04  18.99  
4             1         7         25.0  2017-10-01  2017-10-05  18.99  


In [36]:
# Add last N day's sales, up to 10 days
X_flat['last_1_day_sales'] = 0
X_flat['last_2_day_sales'] = 0
X_flat['last_3_day_sales'] = 0
X_flat['last_4_day_sales'] = 0
X_flat['last_5_day_sales'] = 0
X_flat['last_6_day_sales'] = 0
X_flat['last_7_day_sales'] = 0
X_flat['last_8_day_sales'] = 0
X_flat['last_9_day_sales'] = 0
X_flat['last_10_day_sales'] = 0
X_flat['last_11_day_sales'] = 0
X_flat['last_12_day_sales'] = 0
X_flat['last_13_day_sales'] = 0
X_flat['last_14_day_sales'] = 0
X_flat['last_15_day_sales'] = 0
X_flat['last_16_day_sales'] = 0
X_flat['last_17_day_sales'] = 0
X_flat['last_18_day_sales'] = 0
X_flat['last_19_day_sales'] = 0
X_flat['last_20_day_sales'] = 0
X_flat['last_21_day_sales'] = 0
X_flat['last_22_day_sales'] = 0
X_flat['last_23_day_sales'] = 0
X_flat['last_24_day_sales'] = 0
X_flat['last_25_day_sales'] = 0
X_flat['last_26_day_sales'] = 0
X_flat['last_27_day_sales'] = 0
X_flat['last_28_day_sales'] = 0

In [37]:
for i in range(12824):
    X_flat['last_1_day_sales'].iloc[i*151+1:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+150].as_matrix()
    X_flat['last_2_day_sales'].iloc[i*151+2:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+149].as_matrix()
    X_flat['last_3_day_sales'].iloc[i*151+3:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+148].as_matrix()
    X_flat['last_4_day_sales'].iloc[i*151+4:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+147].as_matrix()
    X_flat['last_5_day_sales'].iloc[i*151+5:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+146].as_matrix()
    X_flat['last_6_day_sales'].iloc[i*151+6:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+145].as_matrix()
    X_flat['last_7_day_sales'].iloc[i*151+7:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+144].as_matrix()
    X_flat['last_8_day_sales'].iloc[i*151+8:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+143].as_matrix()
    X_flat['last_9_day_sales'].iloc[i*151+9:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+142].as_matrix()
    X_flat['last_10_day_sales'].iloc[i*151+10:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+141].as_matrix()
    X_flat['last_11_day_sales'].iloc[i*151+11:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+140].as_matrix()
    X_flat['last_12_day_sales'].iloc[i*151+12:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+139].as_matrix()
    X_flat['last_13_day_sales'].iloc[i*151+13:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+138].as_matrix()
    X_flat['last_14_day_sales'].iloc[i*151+14:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+137].as_matrix()
    X_flat['last_15_day_sales'].iloc[i*151+15:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+136].as_matrix()
    X_flat['last_16_day_sales'].iloc[i*151+16:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+135].as_matrix()
    X_flat['last_17_day_sales'].iloc[i*151+17:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+134].as_matrix()
    X_flat['last_18_day_sales'].iloc[i*151+18:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+133].as_matrix()
    X_flat['last_19_day_sales'].iloc[i*151+19:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+132].as_matrix()
    X_flat['last_20_day_sales'].iloc[i*151+20:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+131].as_matrix()
    X_flat['last_21_day_sales'].iloc[i*151+21:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+130].as_matrix()
    X_flat['last_22_day_sales'].iloc[i*151+22:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+129].as_matrix()
    X_flat['last_23_day_sales'].iloc[i*151+23:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+128].as_matrix()
    X_flat['last_24_day_sales'].iloc[i*151+24:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+127].as_matrix()
    X_flat['last_25_day_sales'].iloc[i*151+25:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+126].as_matrix()
    X_flat['last_26_day_sales'].iloc[i*151+26:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+125].as_matrix()
    X_flat['last_27_day_sales'].iloc[i*151+27:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+124].as_matrix()
    X_flat['last_28_day_sales'].iloc[i*151+28:i*151+151] = Y_flat['sales'].iloc[i*151:i*151+123].as_matrix()
    
    if i%128 == 0:
        print('{} products finished'.format(str(i)))

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0 products finished


128 products finished


256 products finished


384 products finished


...

12288 products finished


12416 products finished


12544 products finished


12672 products finished


12800 products finished


In [40]:
# Check 1 result if implementation above is correct
product = 3022
print(X_flat.iloc[product*151:product*151+151]) # Check values for sampled product
X_flat.iloc[product*151:product*151+151].to_csv('./data/clean/flat_sample_{}.csv'.format(str(product)))

           key  pid_x size_x    color   brand    rrp  mainCategory  category  \
456322  12985L  12985      L  schwarz  adidas  48.19             1         7   
456323  12985L  12985      L  schwarz  adidas  48.19             1         7   
456324  12985L  12985      L  schwarz  adidas  48.19             1         7   
456325  12985L  12985      L  schwarz  adidas  48.19             1         7   
456326  12985L  12985      L  schwarz  adidas  48.19             1         7   
456327  12985L  12985      L  schwarz  adidas  48.19             1         7   
456328  12985L  12985      L  schwarz  adidas  48.19             1         7   
456329  12985L  12985      L  schwarz  adidas  48.19             1         7   
456330  12985L  12985      L  schwarz  adidas  48.19             1         7   
456331  12985L  12985      L  schwarz  adidas  48.19             1         7   
456332  12985L  12985      L  schwarz  adidas  48.19             1         7   
456333  12985L  12985      L  schwarz  a

In [41]:
def is_eleventh(row):
    if row['date'][-2:] == '11':
        return 1
    else: 
        return 0

X_flat['is_eleventh'] = X_flat.apply(is_eleventh, axis=1)

In [42]:
def is_crazy_day(row):
    if row['date'] == '2017-11-24': # Black Friday
        return 1
    return 0

X_flat['is_crazy_day'] = X_flat.apply(is_crazy_day, axis=1)

In [43]:
# One-hot encoding days of week
X_flat['day_of_week'] = pd.to_datetime(X_flat['date']).dt.weekday_name

In [44]:
days = pd.get_dummies(X_flat['day_of_week'], prefix='day')
X_flat = X_flat.join(days)

In [45]:
X_flat['days_since_release'] = (pd.to_datetime(X_flat['date']) - pd.to_datetime(X_flat['releaseDate'])).dt.days

In [46]:
X_flat['price_diff'] = X_flat['price'] - X_flat['rrp']

In [47]:
# One-hot encoding everything else except sub-category (too many of them)
colors = pd.get_dummies(X_flat['color'], prefix='color')
brands = pd.get_dummies(X_flat['brand'], prefix='brand')
main_cats = pd.get_dummies(X_flat['mainCategory'], prefix='maincat')
cats = pd.get_dummies(X_flat['category'], prefix='cat')
sub_cats = pd.get_dummies(X_flat['subCategory'], prefix='subcat')

X_flat = X_flat.join(colors)
X_flat = X_flat.join(brands)
X_flat = X_flat.join(main_cats)
X_flat = X_flat.join(cats)

In [48]:
def marketing_activity(row):
    marketingactivities = ['2017-10-11','2017-10-16','2017-11-04','2017-11-11','2017-11-23','2017-11-24', 
                           '2017-11-25','2017-11-27','2017-12-03','2017-12-27','2017-12-28','2017-12-31', 
                           '2018-01-14','2018-01-22','2018-01-23','2018-01-30','2018-02-06','2018-02-07', 
                           '2018-02-20','2018-02-22','2018-02-23']
    if row['date'] in marketingactivities:
        return 1
    else:
        return 0
    
X_flat['marketing_activity'] = X_flat.apply(marketing_activity, axis=1)

In [49]:
X_flat.columns

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'releaseDate',
       ...
       'cat_7', 'cat_10', 'cat_16', 'cat_18', 'cat_24', 'cat_30', 'cat_33',
       'cat_36', 'cat_37', 'marketing_activity'],
      dtype='object', length=108)

# Dump data files to directory

In [50]:
# Dump to pickle
pickle.dump(X_flat, open('./data/clean/X_flat_final.pkl', 'wb'))
pickle.dump(Y_flat, open('./data/clean/Y_flat_final.pkl', 'wb'))
# Read the file later by: X_flat = pickle.load(open('path/to/X_flat.pkl', "rb"))